In [22]:
import pandas as pd
import numpy as np

In [23]:
df = pd.read_csv("data/uber_data.csv")

In [24]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80


In [25]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   VendorID               100000 non-null  int64         
 1   tpep_pickup_datetime   100000 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  100000 non-null  datetime64[ns]
 3   passenger_count        100000 non-null  int64         
 4   trip_distance          100000 non-null  float64       
 5   pickup_longitude       100000 non-null  float64       
 6   pickup_latitude        100000 non-null  float64       
 7   RatecodeID             100000 non-null  int64         
 8   store_and_fwd_flag     100000 non-null  object        
 9   dropoff_longitude      100000 non-null  float64       
 10  dropoff_latitude       100000 non-null  float64       
 11  payment_type           100000 non-null  int64         
 12  fare_amount            100000 non-null  float

In [27]:
df = df.drop_duplicates().reset_index(drop=True)

In [28]:
import uuid
df['trip_ID'] = [str(uuid.uuid4()) for _ in range(len(df))]

In [29]:
dim_vendor = df[['VendorID']].drop_duplicates().reset_index(drop = True)
dim_vendor['vendor_ID'] = dim_vendor['VendorID']
dim_vendor['vendor_name'] = dim_vendor['VendorID'].map({
1: 'Creative Mobile Technologies', 
2: 'VeriFone'})
dim_vendor = dim_vendor[['vendor_ID', 'vendor_name']]

In [30]:
dim_payment_type = df[['payment_type']].drop_duplicates().reset_index(drop = True)
dim_payment_type['payment_type_ID'] = dim_payment_type['payment_type']
dim_payment_type['payment_description'] = dim_payment_type['payment_type'].map({
    1: 'Credit Card',
    2: 'Cash',
    3: 'No Charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided Trip'}) 
dim_payment_type = dim_payment_type[['payment_type_ID','payment_description']]

In [31]:
dim_ratecode = df[['RatecodeID']].drop_duplicates().reset_index(drop = True)
dim_ratecode['ratecode_ID'] = dim_ratecode['RatecodeID']
dim_ratecode['ratecode_description'] = dim_ratecode['RatecodeID'].map({
    1: 'Standard rate',
    2: 'JFK',
    3: 'Newark',
    4: 'Nassau or Westchester',
    5: 'Negotiated fare',
    6: 'Group ride'})
dim_ratecode = dim_ratecode[['ratecode_ID','ratecode_description']]

In [32]:
dim_pickup_location = df[['pickup_longitude', 'pickup_latitude']].drop_duplicates().reset_index(drop=True)
dim_pickup_location['pickup_location_ID'] = dim_pickup_location.index
dim_pickup_location['zone_name'] = np.random.choice(
    ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'], size=len(dim_pickup_location)
)
dim_pickup_location['borough'] = np.random.choice(
    ['Downtown', 'Midtown', 'Uptown', 'Central'], size=len(dim_pickup_location)
)
dim_pickup_location = dim_pickup_location[['pickup_location_ID', 'pickup_longitude', 'pickup_latitude', 'zone_name', 'borough']]

In [33]:
dim_pickup_location.head()

,pickup_location_ID,pickup_longitude,pickup_latitude,zone_name,borough
0,0,-73.976746,40.765152,Brooklyn,Uptown
1,1,-73.983482,40.767925,Brooklyn,Downtown
2,2,-73.782021,40.644810,Queens,Downtown
3,3,-73.863419,40.769814,Queens,Uptown
4,4,-73.971741,40.792183,Staten Island,Midtown


In [34]:
dim_dropoff_location = df[['dropoff_longitude', 'dropoff_latitude']].drop_duplicates().reset_index(drop=True)
dim_dropoff_location['dropoff_location_ID'] = dim_dropoff_location.index
dim_dropoff_location['zone_name'] = np.random.choice(
    ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'], size=len(dim_dropoff_location)
)
dim_dropoff_location['borough'] = np.random.choice(
    ['Downtown', 'Midtown', 'Uptown', 'Central'], size=len(dim_dropoff_location)
)
dim_dropoff_location = dim_dropoff_location[['dropoff_location_ID', 'dropoff_longitude', 'dropoff_latitude', 'zone_name', 'borough']]

In [35]:
dim_dropoff_location.head()

,dropoff_location_ID,dropoff_longitude,dropoff_latitude,zone_name,borough
0,0,-74.004265,40.746128,Manhattan,Uptown
1,1,-74.005943,40.733166,Bronx,Central
2,2,-73.974541,40.675770,Queens,Uptown
3,3,-73.969650,40.757767,Manhattan,Downtown
4,4,-74.177170,40.695053,Manhattan,Uptown


In [36]:
fact_trip = df[['trip_ID', 'VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
                'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 
                'RatecodeID', 'store_and_fwd_flag', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 
                'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']]

In [37]:
fact_trip = fact_trip.merge(dim_vendor, left_on='VendorID', right_on='vendor_ID', how='left').drop(columns=['VendorID'])
fact_trip = fact_trip.merge(dim_pickup_location, on=['pickup_longitude', 'pickup_latitude'], how='left').drop(columns=['pickup_longitude', 'pickup_latitude'])
fact_trip = fact_trip.merge(dim_dropoff_location, on=['dropoff_longitude', 'dropoff_latitude'], how='left').drop(columns=['dropoff_longitude', 'dropoff_latitude'])
fact_trip = fact_trip.merge(dim_ratecode, left_on='RatecodeID', right_on='ratecode_ID', how='left').drop(columns=['RatecodeID'])
fact_trip = fact_trip.merge(dim_payment_type, left_on='payment_type', right_on='payment_type_ID', how='left').drop(columns=['payment_type'])

In [38]:
fact_trip = fact_trip[['trip_ID', 'vendor_ID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
                       'pickup_location_ID', 'dropoff_location_ID', 'ratecode_ID', 'store_and_fwd_flag', 
                       'payment_type_ID', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 
                       'improvement_surcharge', 'total_amount']]

In [39]:
fact_trip.head()

,trip_ID,vendor_ID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,pickup_location_ID,dropoff_location_ID,ratecode_ID,store_and_fwd_flag,payment_type_ID,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,cb2e73ff-b285-486e-98d9-d6cbedb33b1c,1,2016-03-01,2016-03-01 00:07:55,1,0,0,1,N,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,58200621-7f83-447d-a452-42e84d644f87,1,2016-03-01,2016-03-01 00:11:06,1,1,1,1,N,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,3c0417bb-c40c-4e78-9e84-e8ea9c69abcc,2,2016-03-01,2016-03-01 00:31:06,2,2,2,1,N,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,c11a8ee4-42e2-43ec-b624-41c5fae550c0,2,2016-03-01,2016-03-01 00:00:00,3,3,3,1,N,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,33fb99ff-5d60-43e6-9c8b-2a04efe9b147,2,2016-03-01,2016-03-01 00:00:00,5,4,4,3,N,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80
